In [75]:
# from http://flothesof.github.io/k-means-numpy.html

# necessary imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()

In [76]:
points = []
def init_points(n):
    global points
    points = np.array(np.vstack(((np.random.randn(4*n, 2) * 0.75 + np.array([1, 0])),
                  (np.random.randn(3*n, 2) * 0.25 + np.array([-0.5, 0.5])),
                  (np.random.randn(2*n, 2) * 0.5 + np.array([-0.5, -0.5])))))

In [77]:
def initialize_centroids(points, k):
    """returns k centroids from the initial points"""
    centroids = points.copy()
    np.random.shuffle(centroids)
    return centroids[:k]

In [78]:
def closest_centroid(points, centroids):
    """returns an array containing the index to the nearest centroid for each point"""
    distances = np.sqrt(((points - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

In [79]:

def move_centroids(closest, centroids):
    """returns the new centroids assigned from the points closest to them"""
    global points
    filtered_centroids = []
    new_centroids = np.array([points[closest==k].mean(axis=0) for k in range(centroids.shape[0])])
    for k in range(centroids.shape[0]):
        T = (new_centroids[k] - centroids[k])**2
        max = np.max(np.abs(T))

        epsilon = 0.00002
        last_centroid = (k == centroids.shape[0] - 1 and len(filtered_centroids) == 0)
        if max > epsilon or last_centroid:
            filtered_centroids.append(new_centroids[k])
        else:
            points = points[closest!=k]
            closest = closest[closest!=k]
            
            
    return np.array(filtered_centroids)

In [80]:
from JSAnimation import IPython_display
from matplotlib import animation

# create a simple animation
fig = plt.figure()
ax = plt.axes(xlim=(-4, 4), ylim=(-4, 4))
centroids = initialize_centroids(points, 6)

line, = ax.plot([], [], lw=2)
init_points(100)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    global centroids
    closest = closest_centroid(points, centroids)
    ax.cla()
    ax.scatter(points[:, 0], points[:, 1], c = closest)
    centroids = move_centroids(closest, centroids)
    
    ax.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)
    
    ax.set_xlim([-2.5,4])
    ax.set_ylim([-2.5,2.5])

    return line,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=40, interval=1, blit=True)

TypeError: list indices must be integers, not tuple